# Build a system to auto-catagorize 16s rRNAs, ITS and blastn search

## some constants

### locations

 content | location 
 -------: | :--------
 blast+ folder | `/mnt/d/linux/P/blast/bin/`
 blastn | `/mnt/d/linux/P/blast/bin/blastn`
 ascp | `/mnt/d/linux/.aspera/connect/bin/ascp`
 ~ | `/mnt/d/linux`

### code examples

ascp
* `/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/blast/db/16SMicrobial.tar.gz  ~/W/NCBI/16SMicriobial/`

## download data

### download taxonamy, gss, wgs, est

run with linux
```
/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_gss.accession2taxid.gz  ~/W/NCBI/taxonamy/

/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_wgs.accession2taxid.gz  ~/W/NCBI/taxonamy/

/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_est.accession2taxid.gz  ~/W/NCBI/taxonamy/

/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/taxdump.tar.gz  ~/W/NCBI/taxonamy/
```


### download nt blast database

generate the download code with Python
```
l = ["/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/blast/db/nt.%02d.tar.gz  ~/W/NCBI/nt201803/"%i for i in range(56)]
open("temp.txt","w").write("\n".join(l))
```
Then run the code stored in `temp.txt` in linux commandline.

## 16SrRNAs of Microbial from NCBI

### download 16SMicrobial.tar.gz from ncbi

The link address is ftp://ftp.ncbi.nlm.nih.gov/blast/db/16SMicrobial.tar.gz
Download the file with ascp.

The download code is as below. Run it in a ubuntu terminal.
```
/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/blast/db/16SMicrobial.tar.gz  ~/W/NCBI/16SMicriobial/
```

decompress the file.
```
cd ~/W/NCBI/16SMicriobial/
tar xzf 16SMicrobial.tar.gz
```

This is a database built for blast search. Extract fasta sequences to see how many sequences are there.
```
~/P/blast/bin/blastdbcmd -entry all -db 16SMicrobial -out 16SMicrobial.fasta
```

There are 19,757 sequences, 20,035 rRNA IDs. This should be a very good reference for 16S rRNAs.

### blast Kejing Wang's sequences against 16SMicrobial

`~/P/blast/bin/blastn -db ~/W/NCBI/16SMicriobial/16SMicrobial -query ~/W/MoonNt/20180327KC.fasta -out ~/W/MoonNt/20180329KC.tab -num_threads 7 -max_target_seqs 200 -outfmt '6 qseqid sseqid qcovhsp pident staxids  sscinames sskingdoms length mismatch gapopen qstart qend sstart send evalue bitscore'`

The sseqid contains gi number and accession number, which looks like "gi|343201646|ref|NR_042372.1|". It is because when they built the database, gi number is still included.

the output looks like

Re-build the database so that gi number is excluded.

### Re-built 16SMicrobial Database

get a Taxonomy id <–> sequence id file for 16SMicrobial

```
from Bio import SeqIO
rna16s = set([e.id for e in SeqIO.parse(open("/mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.fasta"),format = "fasta")])
acc2tax = open("/mnt/d/linux/W/NCBI/taxonamy/nucl_gb.accession2taxid")
fout = open("/mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.taxa","w", newline = "\n")
for line in acc2tax:
    _es = line.split()
    if _es[1] in rna16s:
        fout.write(_es[1]+' ' + _es[2] + "\n")
fout.close()
```
checked, all 18757 sequences have taxonomy data available.

build the database

```
~/P/blast/bin/makeblastdb -in /mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.fasta -input_type fasta -dbtype nucl -parse_seqids -title "16SMicrobial" -out /mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial_DB  -taxid_map /mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.taxa
```

### blast Kejing Wang's sequences against newly built 16SMicrobial

`~/P/blast/bin/blastn -db ~/W/NCBI/16SMicriobial/16SMicrobial_DB -query ~/W/MoonNt/20180327KC.fasta -out ~/W/MoonNt/20180329KC.tab_n -num_threads 7 -max_target_seqs 200 -outfmt '6 qseqid sseqid qcovhsp pident staxids  sscinames sskingdoms length mismatch gapopen qstart qend sstart send evalue bitscore'`

The sseqid contains gi number and accession number, which looks like "gi|343201646|ref|NR_042372.1|". It is because when they built the database, gi number is still included.

the output looks like

still not the form we want. Give up this test. Use the original Database.

## generate the species, genus, family, order, class, phylum table

### species, genus, family, order, class, phylum table

```
file_node = "/mnt/d/linux/W/NCBI/taxonamy/nodes.dmp"
import pandas as pd
df_node = pd.read_csv(file_node, sep = "\t",header = None,dtype = str)
df_node = df_node.loc[:,[0,2,4]]
df_node.columns = ["From","To","rank"]
df_node.head()

set(df_node.loc[:,"rank"])

def f(x):
    if x == "subspecies" or x == 'varietas':
        return "species"
    return x

from collections import Counter
print(Counter(df_node["rank"]))
df_node["rank"] = df_node["rank"].apply(f)
print(Counter(df_node["rank"]))



ls_species = list(df_node[df_node["rank"] == "species"].iloc[:,0])

dc_node = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"To"])))
dc_rank = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"rank"])))

## added 20180531 ##
## some 'no rank' rank is actually subspecies. 
def correct_no_rank(x):
    '''
    if rank == 'no rank', check if it is a subspecies. if it is, return 'species'
    '''
    if x['rank'] == 'no rank':
        if dc_rank[x['To']] == 'species':
            return 'species'
        if dc_rank[dc_node[x['To']]] == 'species':
            return 'species'
    return x['rank']

df_node['rank2'] = df_node.apply(correct_no_rank, axis=1)
ls_species = list(df_node[df_node["rank2"] == "species"].iloc[:,0])
## end added 20180531 ##

def find_lineage(species):
    '''
    given a species ID, return a list of species, genus, family, order, class and phylum id
    '''
    lineage = {}
    From = species
    rank_set = {'genus','family','order','class','phylum'}
    lineage['species'] = species
    while(True):
        To = dc_node[From]
        rank = dc_rank[From]
        if From in {"2","2157","12884","10239","4751","33090","33208"}:
            lineage["kingdom"] = From
        if From == To:
            break
        if rank in rank_set:
            lineage[rank] = From
            From = To
        else:
            From = To
    return lineage

ls_lineage = list(map(find_lineage,ls_species))
df_lineage = pd.DataFrame(ls_lineage, dtype=str)
df_lineage = df_lineage.loc[:,['species','genus','family','order','class','phylum',"kingdom"]]

df_lineage.to_csv("/mnt/d/linux/W/NCBI/taxonamy/speicesLineage", sep = "\t", float_format = "%d", index = False)

```

kingdom:
* Life
    * Archae 2157
    * Bacteria 2
    * Viroids 12884
    * Viruses 10239
    * Eukaryota 2759
        * Fungi 4751
        * viridiplantae 33090
        * metazoa 33208

file looks like. For some species, some values are missing.

species | genus | family | order | class | phylum | kingdom
--------|-------|--------|-------|-------|----- |----
7 | 6 | 335928 | 356 | 28211 | 1224 | 2
9 | 32199 | 1903409 | 91347 | 1236 | 1224 | 2
11 | 1707 | 85016 | 85006 | 1760 | 201174 | 2
14 | 13 | 203488 | 203487 | 203486 | 68297 | 2
17 | 16 | 32011 | 32003 | 28216 | 1224 | 2
19 | 18 | 213421 | 69541 | 28221 | 1224 | 2
21 | 20 | 76892 | 204458 | 28211 | 1224 | 2
23 | 22 | 267890 | 135622 | 1236 | 1224 | 2
24 | 22 | 267890 | 135622 | 1236 | 1224 | 2
25 | 22 | 267890 | 135622 | 1236 | 1224 | 2
27 |  |  |  |  |  | 2
28 |  |  |  |  |  | 2


### lineage ID and scientific name

We got the file "speciesLineage" coding in numbers.
Generate a file with lineage ID and scientific names.
```
file_name = "/mnt/d/linux/W/NCBI/taxonamy/names.dmp"
import pandas as pd
df_name = pd.read_csv(file_name,sep = "\t", header = None,dtype = str)

df_name = df_name.loc[:,[0,2,6]]
df_name_keep = df_name[df_name[6] == "scientific name"]

df_name_keep = df_name_keep.loc[:,[0,2]]
df_name_keep.columns = ["id","name"]
df_name_keep.to_csv("/mnt/d/linux/W/NCBI/taxonamy/speciesName", sep = "\t", float_format = "%d", index = False)
```

File looks like:
```
id	name
1	root
2	Bacteria
6	Azorhizobium
7	Azorhizobium caulinodans
9	Buchnera aphidicola
```

### lineage ID and lineage names

change speicesLineage to speicesLineageName, with two column, taxid and species lineage
```

import pandas as pd

file_Name = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
file_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"

df_Lineage = pd.read_csv(file_Lineage,sep="\t",dtype=str)

dcName = {}
for _l in open(file_Name):
    _k,_v = _l[:-1].split("\t")
    dcName[_k] = _v

df_LineageName = df_Lineage.copy()
df_LineageName.index = df_LineageName["species"].copy()
df_LineageName.index.name = "taxID"
df_LineageName = df_LineageName.applymap(lambda x: dcName[x] if x in dcName else "")
df_LineageName.to_csv("/mnt/d/linux/W/NCBI/taxonamy/speicesLineageName",sep="\t")

df_head = df_LineageName.apply(lambda x:";".join(x),axis=1)

open("/mnt/d/linux/W/NCBI/taxonamy/speicesLineage.head","w").write(''.join(df_LineageName.apply(lambda x:x.name+'\t'+';'.join(x)+'\n',axis=1)))

```

### check if all 16SMicrobial have full taxonomy

```
templs = open("/mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.taxa").readlines()
templs = [e.split() for e in templs]
taxa_16S = [e[1] for e in templs]
taxa_16S = set(taxa_16S)
id_16S = [e[0] for e in templs]

fileLineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
ls = open(fileLineage).readlines()
ls = ls[1:]
ls = [e.replace("\n","") for e in ls]
ls = [e.split("\t") for e in ls]
dc = {e[0]:e for e in ls if e.count("") == 0}
print(len([e for e in taxa_16S if e not in dc]))

good_taxa16S = [e for e in taxa_16S if e in dc]
dc_good = {e:dc[e] for e in good_taxa16S}
#dc_good['173959']

ls = open("/mnt/d/linux/W/NCBI/taxonamy/speciesName").readlines()
ls = ls[1:]
ls = [e.replace("\n","") for e in ls]
ls = [e.split("\t") for e in ls]
dcName = {e[0]:e[1] for e in ls}

dc_goodName = {e:[dcName[i] for i in dc_good[e]] for e in dc_good}
fout = open("/mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial.Lineage","w")
fout.write("id\tspecies\tgenus\tfamily\torder\tclass\tphylum\n")
for e in dc_goodName:
    fout.write(e + "\t" + "\t".join(dc_goodName[e]) + "\n")
fout.close()
```

1893 out of 14875 species in 16SMicrobial do not have "perfect" taxonomy terms. 
Generate a file for usage of these 12982 species directly.
The file is saved as "16SMicrobial.Lineage".

## blast against NCBI nt

### blast Kejing Wang's sequences against nt

`~/P/blast/bin/blastn -db ~/W/NCBI/nt/nt -query ~/W/MoonNt/20180327KC.fasta -out ~/W/MoonNt/20180329KC_nt.tab -num_threads 7 -max_target_seqs 200 -outfmt '6 qseqid sseqid qcovhsp pident staxids  sscinames sskingdoms length mismatch gapopen qstart qend sstart send evalue bitscore'`

Just to check the speed of blast nt.

This step is extremely slow. We need to reduce the size of database to speed up the process

## Prepare rRNA database

### extract all ribosome-related sequences

rRNA sequences usually have "ribosomal RNA" or "rRNA" as key words in the headline of the fasta file. To include all ribosome-related sequences, use "ribosom" and "rRNA" as key words to extract them all.

```
filename = "/mnt/d/linux/W/NCBI/nt/nt"
fo = open(filename)
import re
fout = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA","w")
for line in fo:
    if line[0] == ">":
        if re.search("ribosom",line,re.I) or re.search("rRNA",line,re.I):
            fout.write(line)
            rRNA = True
        else:
            rRNA = False
    else:
        if rRNA:
            fout.write(line)
fout.close()
fo.close()
```

This file is too big for hs-blastn. Not enough memory in the 32GB PC. Further extract 16s RNA sequences using other filter parameters

### build a database rRNA, hs-blastn

```
cd /mnt/d/linux/W/NCBI/nt_rRNA/
~/P/blast/bin/makeblastdb -in rRNA -input_type fasta -dbtype nucl
~/P/blast/bin/windowmasker -in rRNA -infmt blastdb -mk_counts -out rRNA.counts
~/P/blast/bin/windowmasker -in rRNA.counts -sformat obinary -out rRNA.counts.obinary -convert
~/P/hs-blastn-src/hs-blastn index rRNA
```

### filter rRNA sequences

 The average lengths of the structural rRNA genes are 1,522 bp, 2,971 bp, and 120 bp respectively for 16S, 23S, and 5S rRNAs.
 Out of 19757 seqs in 16sMicrobial, only 19 is shorter than 1000 bp. 10 is longer than 1800.
 remove sequences with more than ten "N"
 
 **1,964,372** sequences in rRNA2
```
from Bio import SeqIO
lsIn = SeqIO.parse( open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA"), format="fasta")
fout = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2", "w")
for s in lsIn:
    if len(s.seq) >= 1000 and len(s.seq) <= 1800 and (str(s.seq).count("N") + str(s.seq).count("n")) < 10:
        fout.write(">"+s.description+"\n"+str(s.seq)+"\n")
fout.close()
```



```
~/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/16SMicriobial/16SMicrobial -query /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -outfmt 6 -out /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.blast16S.tab -num_alignments 1 -word_size 28 -num_threads 8

```

blastn is slow. Go back to hs-blastn
```
cd /mnt/d/linux/W/NCBI/16SMicriobial/
~/P/hs-blastn-src/hs-blastn align -db 16SMicrobial.fasta -window_masker_db 16SMicrobial.fasta.counts.obinary -query /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -out /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.16S.tab.HS -num_threads 8 -max_target_seqs 1 -outfmt 6 -word_size 28
```

filter those with alignments matched length >800, identity > 80%.

```
import pandas as pd
df = pd.read_csv("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.16S.tab.HS", header = None, sep = "\t")
df1 = df[(df[2] > 80) & (df[3] > 800)]
possible_16S = set(df1[0])

from Bio import SeqIO
fout = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA3", "w")
seqs = SeqIO.parse(open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2"),"fasta")
for seq in seqs:
    if seq.id in possible_16S:
        fout.write(">"+seq.description+"\n"+str(seq.seq)+"\n")
fout.close()
```

1,542,730 sequences in rRNA3 file.

### build a database for filtered rRNA, hs-blastn

```
cd /mnt/d/linux/W/NCBI/nt_rRNA/
~/P/blast/bin/makeblastdb -in rRNA3 -input_type fasta -dbtype nucl
~/P/blast/bin/windowmasker -in rRNA3 -infmt blastdb -mk_counts -out rRNA3.counts
~/P/blast/bin/windowmasker -in rRNA3.counts -sformat obinary -out rRNA3.counts.obinary -convert
~/P/hs-blastn-src/hs-blastn index rRNA3
```

The output of hs-blastn is not good. The input file contains 255 sequences, while the output file contains 158 query ids. This problem cannot be solved by using the recommended blast+2.3.0. Give up all test best on hs-blastn.

### build a database for rRNA2

prepare the taxid_map file. Note, taxid_map, use accession number without the version number.
```
from Bio import SeqIO
seqs = SeqIO.parse(open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2"),format="fasta")
names = set([e.id for e in seqs])
taxa = open("/mnt/d/linux/W/NCBI/taxonamy/nucl_gb.accession2taxid")
taxa_rRNA2 = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.taxa","w")
for line in taxa:
    e1,e2, e3, e4 = line.split()
    if e2 in names:
        taxa_rRNA2.write(e1 + " " + e3 + "\n")
taxa_rRNA2.close()
```
build the database

```
/mnt/d/linux/P/blast/bin/dustmasker -in /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -input_type fasta -dbtype nucl -mask_data /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.asnb -out /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -title "rRNA2" -parse_seqids -taxid_map /mnt/d/linux/W/NCBI/rRNA2.taxa
```

### blast Kejing Wang's seq against nt rRNA3

```
cd /mnt/d/linux/W/NCBI/nt_rRNA/
~/P/hs-blastn-src/hs-blastn align -db rRNA3 -window_masker_db rRNA3.counts.obinary -query ~/W/MoonNt/20180327KC.fasta -out ~/W/MoonNt/20180330KC_ntRNA.tab -num_threads 8 -max_target_seqs 200 -outfmt 6 -word_size 12


~/P/blast/bin/blastn -db rRNA3 -query ~/W/MoonNt/20180327KC.fasta -out ~/W/MoonNt/20180330KC_ntRNA.tab_blastn -num_threads 7 -max_target_seqs 200 -outfmt 6
```

For hs-blastn search, use `-word_size 12` to generate similar result as `blastn`. The speed is very good. The result is still a little different compared to `blastn`. Design the pipleline carefully. 

First, use blastn against 16SMicrobial and hs-blastn against rRNA3. If no significant different difference for the best match, use 16SMicrobial results. Else, run blastn against rRNA3.

### extract sequence length, header, and taxomany id

generate 3 files. rRNA2.len, rRNA2.head, rRNA2.taxa, rRNA2.rare.

```
filename = "/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2"


from Bio import SeqIO
f_silva = filename
seqs = SeqIO.parse(open(f_silva),"fasta")
fout1 = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.len","w")
fout2 = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.head","w")
accs = []
for seq in seqs:
    fout1.write(seq.id + "\t" + str(len(seq.seq)) + "\n")
    fout2.write(seq.id + "\t" +seq.description.split(" ",maxsplit=1)[1] + "\n")
    accs.append(seq.id)
fout1.close()
fout2.close()

accs = set(accs)
accs_v = list(accs)
accs = set([e.split(".")[0] for e in accs_v])
print("accs length", len(accs))

#creat a dictionary with accession number as key, tax id as value
dc_acc2tax = {}

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

accsNotInNCBI = [e for e in accs if e not in dc_acc2tax]
print(len(accsNotInNCBI))

fout = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.taxaRare","w")
for seq_acc in accs:
    if seq_acc in dc_acc2tax:
        fout.write(seq_acc + "\t" + dc_acc2tax[seq_acc] +"\n")
    else:
        fout2.write(seq_acc + "\n")
fout.close()
fout2.close()
```

## SILVA 16S rRNA database

### basic info

SILVA rRNA database project
A comprehensive on-line resource for quality checked and aligned ribosomal RNA sequence data.
I downloaded 16S rRNA database from the link `https://www.arb-silva.de/fileadmin/silva_databases/release_132/Exports/SILVA_132_SSURef_tax_silva.fasta.gz`

There are **2,090,688** sequences. 

The titles of the sequences are `JQ771801.1.1459 Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces sp. Ac105`. 
* `JQ771801` accession number of NCBI
* `1.1459` the location of rRNA in the sequence
* `Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces sp. Ac105`: kingdom;phylum;class;order;family;genus;species

check if all sequences have titles shown above. It turns out the taxonamy is not always listed as above (which contains 7 parts). It can be like `Eukaryota;Opisthokonta;Holozoa;Metazoa (Animalia);Eumetazoa;Bilateria;Chordata;Vertebrata;Gnathostomata;Euteleostomi;Tetrapoda;Mammalia;Homo sapiens (human)`, which means it contains full lineage. 
```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_SSURef_tax_silva.fasta"
seqs = list(SeqIO.parse(open(f_silva),"fasta"))
titles = [e.description for e in seqs]
import re
re.match("^\w*\.\d*\.\d* \w*;([a-zA-Z 0-9]*;){6}$",titles[0])
nogood = [e for e in titles if not re.match("^\w*\.\d*\.\d* \w*;([a-zA-Z 0-9]*;){5}([a-zA-Z 0-9]*)$",e)]
nogood2 = [e for e in titles if not re.match("^\w*\.\d*\.\d* \w*;([^;]*;){5}([a-zA-Z 0-9]*)$",e)]
```


2,090,688 sequences, form **2,040,731** accession numbers. **5244** of them do not have good taxonamy ids.


### convert to DNA

Convert sequences of RNA to DNA.

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_SSURef_tax_silva.fasta"
seqs = SeqIO.parse(open(f_silva),"fasta")
fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.fasta","w")
for seq in seqs:
    fout.write(">" + seq.description + "\n" + str(seq.seq).upper().replace("U","T")+"\n")
fout.close()
```

### extract seq length

Write a file with seq length
`JQ771801.1.1459 1459`

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_SSURef_tax_silva.fasta"
seqs = SeqIO.parse(open(f_silva),"fasta")
fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.len","w")
for seq in seqs:
    fout.write(seq.id + "\t" + str(len(seq.seq)) + "\n")
fout.close()
```

### extract seq header

`JQ771801.1.1459 Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces sp. Ac105`

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_SSURef_tax_silva.fasta"
seqs = SeqIO.parse(open(f_silva),"fasta")
fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.head","w")
for seq in seqs:
    fout.write(seq.id + "\t" +seq.description.split(" ",maxsplit=1)[1] + "\n")
fout.close()
```

### extract taxonamy id

Write a file with seqID, accession number, length, taxonamy. The separater is "\t"
`JQ771801 taxID`. 
If seqID do not have taxID, write another file
`JQ771801 Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces sp. Ac105`

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_SSURef_tax_silva.fasta"
seqs = list(SeqIO.parse(open(f_silva),"fasta"))
dcseqs = {e.id.split(".")[0]:e for e in seqs}
#get the accession ids
accs = set([e.id.split(".")[0] for e in seqs])
print(len(accs))

#get a dictionary of seqs
dcseqs = {e.id.split(".")[0]:e for e in seqs}


#creat a dictionary with accession number as key, tax id as value
dc_acc2tax = {}

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

accsNotInNCBI = [e for e in accs if e not in dc_acc2tax]
print(len(accsNotInNCBI))

fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.taxaRare","w")
for seq_acc in dcseqs:
    if seq_acc in dc_acc2tax:
        fout.write(seq_acc + "\t" + dc_acc2tax[seq_acc] +"\n")
    else:
        fout2.write(seq_acc + "\t" + dcseqs[seq_acc].description.split(" ",maxsplit = 1)[1] + "\n")
fout.close()
fout2.close()


```

### build blast db

```
cd /mnt/d/linux/W/NCBI/SILVA/
/mnt/d/linux/P/blast/bin/dustmasker -in SILVA16sDNA.fasta -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out SILVA16sDNA.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in SILVA16sDNA.fasta -input_type fasta -dbtype nucl -mask_data SILVA16sDNA.asnb -out SILVA16sDNA -title "SILVA16s" -parse_seqids -taxid_map SILVA16sDNA.taxa
```

### build a file for species with good lineage info

speicesLineage, only keep those with five elements per line.
For taxonamy, speciesName, remove those contain the keywords "environment", "unidentified", "uncultured". Generate a file.
of the six terms "species	genus	family	order	class	phylum", allow missing two of them.

Filename **SILVA16sDNA.goodLineage**

```
f_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
ls_Lineage = open(f_Lineage).readlines()

ls_Lineage = [e.split() for e in ls_Lineage]
ls_LineageGood = [e for e in ls_Lineage if len(e) >= 5]
ls_LineageGood2 = [e for e in ls_Lineage if len(e) >= 4]

ls_LineageGoodSpecies = [e[0] for e in ls_LineageGood2]
st_goodSpecies = set(ls_LineageGoodSpecies[1:])

f_names = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
import re
ls_names = open(f_names).readlines()[1:]
ls_namesSpecies = [e for e in ls_names if e.split()[0] in st_goodSpecies]
ls_goodnames = []
noGood = 0
for e in ls_namesSpecies:
    if re.search("environment", e, re.I) or re.search("unidentified", e, re.I) or re.search("uncultured", e, re.I):
        noGood += 1
    else:
        ls_goodnames.append(e)
print(len(ls_goodnames))
print(noGood)

ls_goodSpecies = [e.split()[0] for e in ls_goodnames]
st_goodSpeciesName = set(ls_goodSpecies) #species not with good name and lineage 

ls_speciesSILVA = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.taxa").readlines()
ls_speciesSILVA = [e.split()[1] for e in ls_speciesSILVA]
st_speciesSILVA = set(ls_speciesSILVA)

#check keep SILVA species in st_goodSpeciesName
st_speciesSILVAgoodLineage = st_speciesSILVA & st_goodSpeciesName
#write the lineage info for st_speciesSILVAgoodLineage
fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.goodLineage","w")
fout.write("\t".join(["taxID","species","genus","family","order","class","phylum","kingdom"]) +"\n")
dc_Lineage = {}
lstemp = open(f_Lineage).readlines()[1:]
for e in lstemp:
    es = e[:-1].split("\t")
    dc_Lineage[es[0]] = es
dc_names = {e[:-1].split("\t")[0]:e[:-1].split("\t")[1] for e in ls_names}
dc_names[""] = ""

for e in st_speciesSILVAgoodLineage:
    _l = dc_Lineage[e]
    _lnames = [dc_names[i] for i in _l]
    fout.write("\t".join([e] + _lnames) +"\n")
fout.close()    
```


## build a database with good taxa

For sequences in SILVA and rRNA2, extract those with good lineage Info.   
starting from`species` and have >3 out of 5 terms in`genus, family, order, class, phylum`

Conclusion: not necessary. The "good taxa" database is still pretty large. Use SILVA

```
f_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
import pandas as pd
import re
from Bio import SeqIO
df_lineage = pd.read_csv(f_Lineage,sep = "\t", dtype = str)
df_lineage["taxaCount"] = df_lineage.apply(lambda x:6-sum(x.isnull()), axis = 1)
df_lineageFilter = df_lineage[df_lineage["taxaCount"] >=4]

species = set(df_lineageFilter["species"])

f_names = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
df_names = pd.read_csv(f_names,sep="\t",dtype=str)
df_names["good"] = df_names["name"].apply(lambda e:False if re.search("environment", e, re.I) or re.search("unidentified", e, re.I) or re.search("uncultured", e, re.I) else True)

speciesGoodname = set(df_names[df_names["good"]]["id"])

speciesGoodLineageName = species & speciesGoodname

accessionGood = []
f = "/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.taxa"
for line in open(f):
    _acc, _species = line.split()
    if _species in speciesGoodLineageName:
        accessionGood.append(_acc)
f = "/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2.taxa"
for line in open(f):
    _acc, _species = line.split()
    if _species in speciesGoodLineageName:
        accessionGood.append(_acc)

accessionGoodst = set(accessionGood)

#get fasta with acc in accessionGoodst
dcgoodfasta = {}

f = "/mnt/d/linux/W/NCBI/nt_rRNA/rRNA2"
for _s in SeqIO.parse(open(f),format="fasta"):
    if _s.id.split(".")[0] in accessionGoodst:
        dcgoodfasta[_s.id.split(".")[0]] = _s
print(len(dcgoodfasta))
f = "/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.fasta"
for _s in SeqIO.parse(open(f),format="fasta"):
    if _s.id.split(".")[0] in accessionGoodst:
        dcgoodfasta[_s.id.split(".")[0]] = _s
print(len(dcgoodfasta))


lsgoodfasta = list(dcgoodfasta.values())
runfile("/mnt/c/Users/ATPs/Documents/GitHub/XCProject/fasta/largeFastaFile.py")
lsunique = fasta_uni_keepone(lsgoodfasta)
fout = open("/mnt/d/linux/W/NCBI/nt_rRNA/rRNA_reference","w")
for e in lsunique:
    fout.write(">"+e.description+"\n"+str(e.seq)+"\n")
fout.close()

f = "/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.fasta"
_ls = open_fasta_to_list(f)
print(len(_ls))
_lsuni = fasta_uni_keepone(_ls)
print(len(_lsuni))

```

## build the pipeline 16S

of the 255 sequences for Kejing Wang, 
* 7 of them are identical. 
* After removing sequences which are completely part of the other one, 179 sequences left. 
* If we allow the matched error rate to 0.001, 167 sequences left.
* If we allow the matched error rate to 0.005, 144 sequences left.
* If we allow the matched error rate to 0.010, 130 sequences left.



```
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 30 13:35:56 2018

@author: x
"""

folder = "/mnt/d/linux/W/MoonNt/KC/"
error_rate = 0.005

import glob
import re
import subprocess
import pandas as pd
import numpy as np
from Bio import SeqIO

def getSeqFiles(folder):
    '''
    given a foler name, return a list of all sequence files.
    seq files are files end with .seq
    '''
    files = glob.glob(folder + '**/*.seq', recursive=True)
    return files

def getFastaFromFile(filename):
    '''
    given a filename, return a str of fasta format.
    some files contain extra info
    filename looks like '/mnt/d/linux/W/MoonNt/KC/180223 SEQ/MN210615.seq'
    return a string of fasta format
    '''
    fasta_str = ">" + re.split("\\\\|/",filename)[-1].split(".")[0] + "\n"
    txt_ls = open(filename).read().split("\n")
    if txt_ls[0][0] != ">":
        seq = [e for e in txt_ls if re.match("[ATCGN]+\n?$",e,re.I)]
        if len(seq) == 0:
            print(filename, "something is wrong")
            return ""
        return fasta_str + ''.join(seq) + "\n"
    return fasta_str + str(SeqIO.read(open(filename),format = "fasta").seq) + "\n"

def taxaFinder(folder = folder):
    '''
    given a path of a folder, output a file with the alignment and taxa info for the sequences
    '''
    # add the slash to make the path easier to use
    if folder[-1] != "/":
        folder = folder + "/"
    
    # get paths of all sequence files
    files = getSeqFiles(folder)
    
    # get all fasta sequences
    fasta_seqs = [getFastaFromFile(e) for e in files]
    
    # save the fasta_seqs to a file in folder
    open(folder + "AllSeqs.fasta","w").write("".join(fasta_seqs))
    
    # run local blast against SILVA
    command_line = "/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA -query {folder}AllSeqs.fasta  -out {folder}AllSeqs.fasta.SILVA.tab -num_threads 8 -max_target_seqs 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'".format(folder = folder)
    subprocess.run(command_line, shell=True)
    
    # read in blast result for SILVA
    df_SILVA = pd.read_csv("{folder}AllSeqs.fasta.SILVA.tab".format(folder = folder), sep = "\t",header = None, dtype = {0:str,1:str,2:float,3:int,4:int,5:int,6:int,7:int,8:int,9:int,10:float,11:float,12:int,13:str})
    df_SILVA.columns = ["query", "subject","identity","matchLength","missMatch","gap","qstart","qend","sstart", "send", "evalue", "bitScore", "qcover", "taxID"]
    ## add column 14, length of identical bases
    df_SILVA["identical"] = round(df_SILVA["identity"] * df_SILVA["matchLength"]/100)
    ## for subject id, only keep accession number
    df_SILVA["subject"] = df_SILVA["subject"].apply(lambda x: x.split(".")[0])
    ## sort by query id and identical bases
#    df_SILVA = df_SILVA.sort_values(by = [0,14],axis = 0, ascending=[True, False])
    ## get max identical bases for each query
#    query_max = df_SILVA.groupby(by = [0])[14].max()
#    query_max = query_max.to_dict()
#    ## add column 15, allow the identical bases to with the error of 0.5% compare to query_max
#    df_SILVA[15] = df_SILVA.apply(lambda x:(query_max[x[0]] - x[14])/query_max[x[0]] <= error_rate, axis = 1)
#    ## identical bases True 
#    df_SILVA_filter1 = df_SILVA[df_SILVA[15]]
    
    ## add a column, to filter identity > 97, qcover > 95
    df_SILVA["filter1"] = (df_SILVA["identity"] > 97) & (df_SILVA["qcover"] > 95)
    
    df_Lineage = pd.read_csv("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.head", sep = "\t", header = None, dtype = str)
    df_Lineage.columns = ["acc","SILVAtaxa"]
    df_Lineage["acc"] = df_Lineage["acc"].apply(lambda x:x.split(".")[0])
    df_Lineage.index = df_Lineage["acc"]
    df_Lineage = df_Lineage.drop(labels=["acc"], axis = 1)
    
    df_SILVA = df_SILVA.join(df_Lineage, on="subject",how="left")
    
    
    # read in SILVA16sDNA.goodLineage as dataframe
    df_goodLineage = pd.read_csv("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.goodLineage", sep = "\t", dtype = str)
    df_goodLineage.index = df_goodLineage["taxID"]
    df_goodLineage = df_goodLineage.drop(labels=["taxID"], axis = 1)
    
    df_SILVA = df_SILVA.join(df_goodLineage, on="taxID",how="left")
    df_SILVA.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover',
       'taxID', 'identical', 'species', 'genus',
       'family', 'order', 'class', 'phylum', 'SILVAtaxa']].to_excel(folder + "AllSILVAresult.xlsx", index = False)
    
    df_SILVA_keep1 = df_SILVA[df_SILVA["filter1"] & df_SILVA["species"] & df_SILVA["taxID"]].drop_duplicates(subset = ["query"], keep = "first").copy()
    
    df_SILVA_keep1.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover',
       'taxID', 'identical', 'species', 'genus',
       'family', 'order', 'class', 'phylum', 'SILVAtaxa']].to_excel(folder + "BestSILVAresult.xlsx", index = False)
    
    queries = set(df_SILVA_keep1["query"])
    fout = open(folder + "sequencesNotIdentified.fasta","w")
    for seq in fasta_seqs:
        _id = seq.split()[0][1:]
        if _id not in queries:
            fout.write(seq)
    fout.close()
    
#    command_line = "/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -query {folder}AllSeqs.fasta  -out {folder}AllSeqs.fasta.NCBI.tab -num_threads 8 -max_target_seqs 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'".format(folder = folder)
#    subprocess.run(command_line, shell=True)


```

## SILVA 18s rRNA database

### basic info

SILVA rRNA database project
A comprehensive on-line resource for quality checked and aligned ribosomal RNA sequence data.
I downloaded 18S rRNA database from the link `https://www.arb-silva.de/fileadmin/silva_databases/release_132/Exports/SILVA_132_LSURef_tax_silva.fasta.gz`

There are **150,865** sequences. 

### convert to DNA, extract seq length, extract seq head

Convert sequences of RNA to DNA.

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_LSURef_tax_silva.fasta"
seqs = list(SeqIO.parse(open(f_silva),"fasta"))

fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.fasta","w")
for seq in seqs:
    fout.write(">" + seq.description + "\n" + str(seq.seq).upper().replace("U","T")+"\n")
fout.close()

fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.len","w")
for seq in seqs:
    fout.write(seq.id + "\t" + str(len(seq.seq)) + "\n")
fout.close()

fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.head","w")
for seq in seqs:
    fout.write(seq.id + "\t" +seq.description.split(" ",maxsplit=1)[1] + "\n")
fout.close()
```

### extract taxonamy id

Write a file with seqID, accession number, length, taxonamy. The separater is "\t"
`JQ771801 taxID`. 
If seqID do not have taxID, write another file
`JQ771801 Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces sp. Ac105`

```
from Bio import SeqIO
f_silva = "/mnt/d/linux/W/NCBI/SILVA/SILVA_132_LSURef_tax_silva.fasta"
seqs = list(SeqIO.parse(open(f_silva),"fasta"))
dcseqs = {e.id.split(".")[0]:e for e in seqs}
#get the accession ids
accs = set([e.id.split(".")[0] for e in seqs])
print(len(accs))

#get a dictionary of seqs
dcseqs = {e.id.split(".")[0]:e for e in seqs}


#creat a dictionary with accession number as key, tax id as value
dc_acc2tax = {}

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

accsNotInNCBI = [e for e in accs if e not in dc_acc2tax]
print(len(accsNotInNCBI))

fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.taxaRare","w")
for seq_acc in dcseqs:
    if seq_acc in dc_acc2tax:
        fout.write(seq_acc + "\t" + dc_acc2tax[seq_acc] +"\n")
    else:
        fout2.write(seq_acc + "\t" + dcseqs[seq_acc].description.split(" ",maxsplit = 1)[1] + "\n")
fout.close()
fout2.close()


```

### build blast db

```
cd /mnt/d/linux/W/NCBI/SILVA/
/mnt/d/linux/P/blast/bin/dustmasker -in SILVA18sDNA.fasta -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out SILVA18sDNA.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in SILVA18sDNA.fasta -input_type fasta -dbtype nucl -mask_data SILVA18sDNA.asnb -out SILVA18sDNA -title "SILVA16s" -parse_seqids -taxid_map SILVA18sDNA.taxa
```

### build a file for species with good lineage info

speicesLineage, only keep those with five elements per line.
For taxonamy, speciesName, remove those contain the keywords "environment", "unidentified", "uncultured". Generate a file.
of the six terms "species	genus	family	order	class	phylum", allow missing two of them.

Filename **SILVA18sDNA.goodLineage**

```
f_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
ls_Lineage = open(f_Lineage).readlines()

ls_Lineage = [e.split() for e in ls_Lineage]
ls_LineageGood = [e for e in ls_Lineage if len(e) >= 5]
ls_LineageGood2 = [e for e in ls_Lineage if len(e) >= 4]

ls_LineageGoodSpecies = [e[0] for e in ls_LineageGood2]
st_goodSpecies = set(ls_LineageGoodSpecies[1:])

f_names = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
import re
ls_names = open(f_names).readlines()[1:]
ls_namesSpecies = [e for e in ls_names if e.split()[0] in st_goodSpecies]
ls_goodnames = []
noGood = 0
for e in ls_namesSpecies:
    if re.search("environment", e, re.I) or re.search("unidentified", e, re.I) or re.search("uncultured", e, re.I):
        noGood += 1
    else:
        ls_goodnames.append(e)
print(len(ls_goodnames))
print(noGood)

ls_goodSpecies = [e.split()[0] for e in ls_goodnames]
st_goodSpeciesName = set(ls_goodSpecies) #species not with good name and lineage 

ls_speciesSILVA = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.taxa").readlines()
ls_speciesSILVA = [e.split()[1] for e in ls_speciesSILVA]
st_speciesSILVA = set(ls_speciesSILVA)

#check keep SILVA species in st_goodSpeciesName
st_speciesSILVAgoodLineage = st_speciesSILVA & st_goodSpeciesName
#write the lineage info for st_speciesSILVAgoodLineage
fout = open("/mnt/d/linux/W/NCBI/SILVA/SILVA18sDNA.goodLineage","w")
fout.write("\t".join(["taxID","species","genus","family","order","class","phylum","kingdom"]) +"\n")
dc_Lineage = {}
lstemp = open(f_Lineage).readlines()[1:]
for e in lstemp:
    es = e[:-1].split("\t")
    dc_Lineage[es[0]] = es
dc_names = {e[:-1].split("\t")[0]:e[:-1].split("\t")[1] for e in ls_names}
dc_names[""] = ""

for e in st_speciesSILVAgoodLineage:
    _l = dc_Lineage[e]
    _lnames = [dc_names[i] for i in _l]
    fout.write("\t".join([e] + _lnames) +"\n")
fout.close()    
```


## Fungi ITS

### basic info

fasta files were downloaded from `https://unite.ut.ee/repository.php`

The header line looks like `Flagelloscypha_japonica|LC146734|SH497095.07FU|reps_singleton| k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Tricholomataceae;g__Flagelloscypha;s__Flagelloscypha_japonica`.

check if all headers have 4 "|" and 7 "__" in name. The answer is Yes.

    unique # of the  0 element split by |  20543
    unique # of the  1 element split by |  58017
    unique # of the  2 element split by |  58048
    unique # of the  3 element split by |  4
Of the four elements the third one is unique, like "SH497095.07FU". Checked some of the 31 accession numbers existing twice in the database, the sequences are identical. Use the accession id as sequence name.  


HM123225 = GenBank/UNITE accession number of representative sequence  
SH221352.07FU = species hypothesis accession number  
UCL7_006587 = compound cluster accession number  
refs = this is a manually designated RefS  
(reps = this is an automatically chosen RepS)  
This specifies the hierarchical classification of the sequence. k = kingdom; p = phylum ; c = class ; o = order ; f = family ; g = genus ; and s = species. Missing information is indicated as "unidentified" item; “f__unidentified;” means that no family name for the sequence exists.

Extract head. Keep similar format like SILVA.
`SH013290.07FU	Fungi;Ascomycota;Leotiomycetes;unidentified;unidentified;unidentified;Leotiomycetes_sp`

Extract goodLineage.  
`
taxID	species	genus	family	order	class	phylum
477	Moraxella lacunata	Moraxella	Moraxellaceae	Pseudomonadales	Gammaproteobacteria	Proteobacteria
`
Here, get taxID from accession ids.

```
from Bio import SeqIO
filename = "/mnt/d/linux/W/NCBI/fungi/sh_general_release_dynamic_s_01.12.2017_dev.fasta"
ls = list(SeqIO.parse(open(filename),"fasta"))

# check if all headers have 4 "|" and 7 "__" in name
print("number of |",set([e.id.count("|") for e in ls]))
print("number of __", set([e.id.count("__") for e in ls]))

for i in range(4):
    print("unique # of the ",i,"element split by | ", len(set([e.id.split("|")[i] for e in ls])))

# get accession numbers
accs = [e.id.split("|")[1] for e in ls]
from collections import Counter
accs_counter = Counter(accs)
accs_counterMorethan1 = {e:accs_counter[e] for e in accs_counter if accs_counter[e] > 1}

dc = {e.id.split("|")[1]:e for e in ls}

#save the fasta file
fout = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.fasta","w")
for _e,e in dc.items():
    fout.write(">"+e.id.split("|")[1]+"\n"+str(e.seq)+"\n")
fout.close()

# extract head
import re
fout = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.head","w")
for e in ls:
    _es = e.id.split("|")
    _acc = _es[1]
    _lineage = re.sub("\w__", "", _es[-1])
    _lineage = _lineage.replace("_"," ")
    fout.write(_acc + "\t" + _lineage + "\n")
fout.close()

# get taxid
accs = set(accs)
#creat a dictionary with accession number as key, tax id as value
dc_acc2tax = {}

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

accsNotInNCBI = [e for e in accs if e not in dc_acc2tax]
print(len(accsNotInNCBI))

fout = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.taxaRare","w")
for seq_acc in dc:
    if seq_acc in dc_acc2tax:
        fout.write(seq_acc + "\t" + dc_acc2tax[seq_acc] +"\n")
    else:
        fout2.write(seq_acc + "\n")
fout.close()
fout2.close()
```

### build blast db
```
cd /mnt/d/linux/W/NCBI/fungi/
/mnt/d/linux/P/blast/bin/dustmasker -in ITS7.2.fasta -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out ITS7.2.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in ITS7.2.fasta -input_type fasta -dbtype nucl -mask_data ITS7.2.asnb -out ITS7.2 -title "ITS7.2" -parse_seqids -taxid_map ITS7.2.taxa
```

### build a file for species with good lineage info

speicesLineage, only keep those with five elements per line.
For taxonamy, speciesName, remove those contain the keywords "environment", "unidentified", "uncultured". Generate a file.
of the six terms "species	genus	family	order	class	phylum", allow missing two of them.

Filename **ITS7.2.goodLineage**

```
f_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
ls_Lineage = open(f_Lineage).readlines()

ls_Lineage = [e.split() for e in ls_Lineage]
ls_LineageGood = [e for e in ls_Lineage if len(e) >= 5]
ls_LineageGood2 = [e for e in ls_Lineage if len(e) >= 4]

ls_LineageGoodSpecies = [e[0] for e in ls_LineageGood2]
st_goodSpecies = set(ls_LineageGoodSpecies[1:])

f_names = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
import re
ls_names = open(f_names).readlines()[1:]
ls_namesSpecies = [e for e in ls_names if e.split()[0] in st_goodSpecies]
ls_goodnames = []
noGood = 0
for e in ls_namesSpecies:
    if re.search("environment", e, re.I) or re.search("unidentified", e, re.I) or re.search("uncultured", e, re.I):
        noGood += 1
    else:
        ls_goodnames.append(e)
print(len(ls_goodnames))
print(noGood)

ls_goodSpecies = [e.split()[0] for e in ls_goodnames]
st_goodSpeciesName = set(ls_goodSpecies) #species not with good name and lineage 

ls_speciesSILVA = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.taxa").readlines()
ls_speciesSILVA = [e.split()[1] for e in ls_speciesSILVA]
st_speciesSILVA = set(ls_speciesSILVA)

#check keep SILVA species in st_goodSpeciesName
st_speciesSILVAgoodLineage = st_speciesSILVA & st_goodSpeciesName
#write the lineage info for st_speciesSILVAgoodLineage
fout = open("/mnt/d/linux/W/NCBI/fungi/ITS7.2.goodLineage","w")
fout.write("\t".join(["taxID","species","genus","family","order","class","phylum","kingdom"]) +"\n")
dc_Lineage = {}
lstemp = open(f_Lineage).readlines()[1:]
for e in lstemp:
    es = e[:-1].split("\t")
    dc_Lineage[es[0]] = es
dc_names = {e[:-1].split("\t")[0]:e[:-1].split("\t")[1] for e in ls_names}
dc_names[""] = ""

for e in st_speciesSILVAgoodLineage:
    _l = dc_Lineage[e]
    _lnames = [dc_names[i] for i in _l]
    fout.write("\t".join([e] + _lnames) +"\n")
fout.close()   
```


## Ezbio

### basic info

**62,240** seqs. 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5563544/

Introducing EzBioCloud: a taxonomically united database of 16S rRNA gene sequences and whole-genome assemblies

Since everything is well prepared for this database. Build the database directly.

**14,849** of them have taxonamy ids, about 25%.

Get the taxa ids for those sequences
```
import pandas as pd
from collections import defaultdict

filename = "/mnt/d/linux/W/NCBI/Ezbio/eztaxon_qiime_full.fasta"
from Bio import SeqIO
lsEzfasta = list(SeqIO.parse(open(filename),"fasta"))

dcName2tax = defaultdict()
with open("/mnt/d/linux/W/NCBI/taxonamy/names.dmp") as f:
    _ls = f.readlines()
for _l in _ls:
    _e = _l.split("\t")
    _k = _e[2]
    _k = _k.upper()
    _v = _e[0]
    dcName2tax[_k] = _v
#    break

filename = "/mnt/d/linux/W/NCBI/Ezbio/eztaxon_id_taxonomy.txt"
with open(filename) as f:
    ls3 = f.readlines()
dc_Ez = {}
for _l in ls3:
    _k,_v = _l.split()
    _v = _v.split(";")[-1].replace("_"," ").upper()
    dc_Ez[_v] = _k

df_Ez = pd.DataFrame(data = {"name":list(dc_Ez.keys()),"seqid":list(dc_Ez.values())})

df_Ez["taxID"] = df_Ez["name"].apply(lambda x:dcName2tax.get(x))

df_Ez_tax = df_Ez[df_Ez["taxID"].notnull()]
df_Ez_noTax = df_Ez[df_Ez["taxID"].isnull()]

df_Ez_tax.to_csv("/mnt/d/linux/W/NCBI/Ezbio/eztaxon_qiime_full.taxa", sep="\t", columns=["seqid","taxID"], header=False, index=False)
df_Ez_noTax.to_csv("/mnt/d/linux/W/NCBI/Ezbio/NoTax", sep="\t")

df_goodLineage = pd.read_csv("/mnt/d/linux/W/NCBI/taxonamy/speicesLineageName", sep = "\t", dtype = str)
df_goodLineage.index = df_goodLineage["taxID"]
df_goodLineage = df_goodLineage.drop(labels=["taxID"], axis = 1)
df_temp = df_Ez_tax.copy()
df_temp = df_temp.join(df_goodLineage,on='taxID',how='left')
df_temp.to_csv("/mnt/d/linux/W/NCBI/Ezbio/eztaxon_qiime_full.goodLineage", sep="\t", columns=['taxID', 'species', 'genus', 'family', 'order','class', 'phylum', 'kingdom'], header=True, index=False)

```

add lineage info for '85620 264669 99810 497816' manually.

### build blastdb
```
cd /mnt/d/linux/W/NCBI/Ezbio/
/mnt/d/linux/P/blast/bin/dustmasker -in eztaxon_qiime_full.fasta -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out eztaxon_qiime_full.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in eztaxon_qiime_full.fasta -input_type fasta -dbtype nucl -mask_data eztaxon_qiime_full.asnb -out eztaxon_qiime_full -title "Ezbio" -parse_seqids -taxid_map eztaxon_qiime_full.taxa
```

### build the pipeline
An updated pipeline with choices for db
```
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 30 13:35:56 2018

@author: x
"""

folder = "/mnt/d/linux/W/MoonNt/KC/"
error_rate = 0.005

import glob
import re
import subprocess
import pandas as pd
import numpy as np
from Bio import SeqIO

def getSeqFiles(folder):
    '''
    given a foler name, return a list of all sequence files.
    seq files are files end with .seq
    '''
    files = glob.glob(folder + '**/*.seq', recursive=True)
    return files

def getFastaFromFile(filename):
    '''
    given a filename, return a str of fasta format.
    some files contain extra info
    filename looks like '/mnt/d/linux/W/MoonNt/KC/180223 SEQ/MN210615.seq'
    return a string of fasta format
    '''
    fasta_str = ">" + re.split("\\\\|/",filename)[-1].split(".")[0] + "\n"
    txt_ls = open(filename).read().split("\n")
    if txt_ls[0][0] != ">":
        seq = [e for e in txt_ls if re.match("[ATCGN]+\n?$",e,re.I)]
        if len(seq) == 0:
            print(filename, "something is wrong")
            return ""
        return fasta_str + ''.join(seq) + "\n"
    return fasta_str + str(SeqIO.read(open(filename),format = "fasta").seq) + "\n"

def taxaFinder(folder = folder):
    '''
    given a path of a folder, output a file with the alignment and taxa info for the sequences
    '''
    # add the slash to make the path easier to use
    if folder[-1] != "/":
        folder = folder + "/"
    
    # get paths of all sequence files
    files = getSeqFiles(folder)
    
    # get all fasta sequences
    fasta_seqs = [getFastaFromFile(e) for e in files]
    folder = folder + folder.split("/")[-2]+"_"
    # save the fasta_seqs to a file in folder
    open(folder + "AllSeqs.fasta","w").write("".join(fasta_seqs))
    
    # run local blast against SILVA
    command_line = "/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA -query {folder}AllSeqs.fasta  -out {folder}AllSeqs.fasta.SILVA.tab -num_threads 8 -max_target_seqs 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'".format(folder = folder)
    subprocess.run(command_line, shell=True)
    
    # read in blast result for SILVA
    df_SILVA = pd.read_csv("{folder}AllSeqs.fasta.SILVA.tab".format(folder = folder), sep = "\t",header = None, dtype = {0:str,1:str,2:float,3:int,4:int,5:int,6:int,7:int,8:int,9:int,10:float,11:float,12:int,13:str})
    df_SILVA.columns = ["query", "subject","identity","matchLength","missMatch","gap","qstart","qend","sstart", "send", "evalue", "bitScore", "qcover", "taxID"]
    ## add column 14, length of identical bases
    df_SILVA["identical"] = round(df_SILVA["identity"] * df_SILVA["matchLength"]/100)
    ## for subject id, only keep accession number
    df_SILVA["subject"] = df_SILVA["subject"].apply(lambda x: x.split(".")[0])
    ## sort by query id and identical bases
#    df_SILVA = df_SILVA.sort_values(by = [0,14],axis = 0, ascending=[True, False])
    ## get max identical bases for each query
#    query_max = df_SILVA.groupby(by = [0])[14].max()
#    query_max = query_max.to_dict()
#    ## add column 15, allow the identical bases to with the error of 0.5% compare to query_max
#    df_SILVA[15] = df_SILVA.apply(lambda x:(query_max[x[0]] - x[14])/query_max[x[0]] <= error_rate, axis = 1)
#    ## identical bases True 
#    df_SILVA_filter1 = df_SILVA[df_SILVA[15]]
    
    ## add a column, to filter identity > 97, qcover > 95
    df_SILVA["filter1"] = (df_SILVA["identity"] > 97) & (df_SILVA["qcover"] > 95)
    
    df_Lineage = pd.read_csv("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.head", sep = "\t", header = None, dtype = str)
    df_Lineage.columns = ["acc","SILVAtaxa"]
    df_Lineage["acc"] = df_Lineage["acc"].apply(lambda x:x.split(".")[0])
    df_Lineage.index = df_Lineage["acc"]
    df_Lineage = df_Lineage.drop(labels=["acc"], axis = 1)
    
    df_SILVA = df_SILVA.join(df_Lineage, on="subject",how="left")
    
    
    # read in SILVA16sDNA.goodLineage as dataframe
    df_goodLineage = pd.read_csv("/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.goodLineage", sep = "\t", dtype = str)
    df_goodLineage.index = df_goodLineage["taxID"]
    df_goodLineage = df_goodLineage.drop(labels=["taxID"], axis = 1)
    
    df_SILVA = df_SILVA.join(df_goodLineage, on="taxID",how="left")
    df_SILVA.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover',
       'taxID', 'identical', 'species', 'genus',
       'family', 'order', 'class', 'phylum', 'SILVAtaxa']].to_excel(folder + "AllSILVAresult.xlsx", index = False)
    
    df_SILVA_keep1 = df_SILVA[df_SILVA["filter1"] & df_SILVA["species"] & df_SILVA["taxID"]].drop_duplicates(subset = ["query"], keep = "first").copy()
    
    df_SILVA_keep1.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover',
       'taxID', 'identical', 'species', 'genus',
       'family', 'order', 'class', 'phylum', 'SILVAtaxa']].to_excel(folder + "BestSILVAresult.xlsx", index = False)
    
    queries = set(df_SILVA_keep1["query"])
    fout = open(folder + "sequencesNotIdentified.fasta","w")
    for seq in fasta_seqs:
        _id = seq.split()[0][1:]
        if _id not in queries:
            fout.write(seq)
    fout.close()
    
#    command_line = "/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/nt_rRNA/rRNA2 -query {folder}AllSeqs.fasta  -out {folder}AllSeqs.fasta.NCBI.tab -num_threads 8 -max_target_seqs 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'".format(folder = folder)
#    subprocess.run(command_line, shell=True)

def taxaFinderEzbio(folder = folder):
    '''
    given a path of a folder, output a file with the alignment and taxa info for the sequences
    '''
    # add the slash to make the path easier to use
    if folder[-1] != "/":
        folder = folder + "/"
    
    # get paths of all sequence files
    files = getSeqFiles(folder)
    
    # get all fasta sequences
    fasta_seqs = [getFastaFromFile(e) for e in files]
    folder = folder + folder.split("/")[-2]+"_"
    # save the fasta_seqs to a file in folder
    open(folder + "AllSeqs.fasta","w").write("".join(fasta_seqs))
    
    # run local blast against SILVA
    command_line = "/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/Ezbio/eztaxon_qiime_full -query {folder}AllSeqs.fasta  -out {folder}AllSeqs.fasta.Ezbio.tab -num_threads 8 -max_target_seqs 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'".format(folder = folder)
    subprocess.run(command_line, shell=True)
    
    # read in blast result for SILVA
    df_Ezbio = pd.read_csv("{folder}AllSeqs.fasta.Ezbio.tab".format(folder = folder), sep = "\t",header = None, dtype = {0:str,1:str,2:float,3:int,4:int,5:int,6:int,7:int,8:int,9:int,10:float,11:float,12:int,13:str})
    df_Ezbio.columns = ["query", "subject","identity","matchLength","missMatch","gap","qstart","qend","sstart", "send", "evalue", "bitScore", "qcover", "taxID"]
#    df_Ezbio['taxID'] = df_Ezbio["subject"]
    ## add column 14, length of identical bases
    df_Ezbio["identical"] = round(df_Ezbio["identity"] * df_Ezbio["matchLength"]/100)
    
    ## add a column, to filter identity > 97, qcover > 95
    df_Ezbio["filter1"] = (df_Ezbio["identity"] > 97) & (df_Ezbio["qcover"] > 95)
    
    df_Lineage = pd.read_csv("/mnt/d/linux/W/NCBI/Ezbio/eztaxon_id_taxonomy.txt", sep = "\t", header = None, dtype = str)
    df_Lineage.columns = ["acc","EzbioTaxa"]
    df_Lineage["EzbioTaxa"] = df_Lineage["EzbioTaxa"].apply(lambda x:x.replace("_"," "))
    df_Lineage.index = df_Lineage["acc"]
    df_Lineage = df_Lineage.drop(labels=["acc"], axis = 1)
    
    df_Ezbio = df_Ezbio.join(df_Lineage, on="subject",how="left")
    
    
    # read in SILVA16sDNA.goodLineage as dataframe
    df_goodLineage = df_Lineage.copy()
    _l = df_goodLineage["EzbioTaxa"].str.split(";")
    df_goodLineage["kingdom"], df_goodLineage["phylum"], df_goodLineage["class"], df_goodLineage["order"], df_goodLineage["family"],  df_goodLineage["genus"], df_goodLineage["species"] = _l.str
    df_goodLineage = df_goodLineage.drop(labels=["EzbioTaxa"], axis = 1)
    
    df_Ezbio = df_Ezbio.join(df_goodLineage, on="subject",how="left")
    df_Ezbio.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover',
       'taxID', 'identical', 'species', 'genus',
       'family', 'order', 'class', 'phylum', 'EzbioTaxa']].to_excel(folder + "AllEzbioResult.xlsx", index = False)
    
    df_Ezbio_keep1 = df_Ezbio[df_Ezbio["filter1"] & df_Ezbio["species"] & df_Ezbio["taxID"]].drop_duplicates(subset = ["query"], keep = "first").copy()
    
    df_Ezbio_keep1.loc[:,['query', 'subject', 'identity', 'matchLength', 'qcover','taxID', 'identical', 'species', 'genus','family', 'order', 'class', 'phylum', 'EzbioTaxa']].to_excel(folder + "BestEzbioResult.xlsx", index = False)
    
    queries = set(df_Ezbio_keep1["query"])
    fout = open(folder + "sequencesNotIdentifiedEzbio.fasta","w")
    for seq in fasta_seqs:
        _id = seq.split()[0][1:]
        if _id not in queries:
            fout.write(seq)
    fout.close()

import argparse
parser = argparse.ArgumentParser(description='the folder path of sequencing results')
parser.add_argument('folder',help = 'input folder path of sequencing results')
parser.add_argument("--db", "-d", help = "database to use, default SILVA", default = "SILVA")
f = parser.parse_args()
folder = f.folder
if f.db == "SILVA":
    taxaFinder(folder)
elif f.db == "Ezbio":
    taxaFinderEzbio(folder)
else:
    print("undefined database")
```

## nt reference with sequences with good lineage

### extract taxid from nt

nt.tax, accession and tax id
```
f_nt = "/mnt/d/linux/W/NCBI/nt/nt"
from Bio import SeqIO
seqs = SeqIO.parse(open(f_nt), "fasta")
accs = []
for seq in seqs:
    accs.append(seq.id.split(".")[0])
print(len(accs))

# get taxid
#accs = set(accs)
#creat a dictionary with accession number as key, tax id as value
dc_acc2tax = dict.fromkeys(accs)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
for l in fo:
    es = l.split()
    if es[0] in accs:
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax))


fout = open("/mnt/d/linux/W/NCBI/nt/nt.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/nt/nt.taxaRare","w")
for _k,_v in dc_acc2tax.items():
    if _v:
        fout.write(_k + "\t" + _v +"\n")
    else:
        fout2.write(_k + "\n")
fout.close()
fout2.close()
```

### extract sequences with good taxonamy

Only include those from kingdoms of Archae, Bacteria, Viroids, Viruses and Fungi.

Also, get **goodLineage**, length.

```

f_Lineage = "/mnt/d/linux/W/NCBI/taxonamy/speicesLineage"
import pandas as pd
import re
from Bio import SeqIO
df_lineage = pd.read_csv(f_Lineage,sep = "\t", dtype = str)
df_lineage["taxaCount"] = df_lineage.apply(lambda x:7-sum(x.isnull()), axis = 1)
df_lineageFilter = df_lineage[df_lineage["taxaCount"] >=5]

species = set(df_lineageFilter["species"])

f_names = "/mnt/d/linux/W/NCBI/taxonamy/speciesName"
df_names = pd.read_csv(f_names,sep="\t",dtype=str)
df_names["good"] = df_names["name"].apply(lambda e:False if re.search("environment", e, re.I) or re.search("unidentified", e, re.I) or re.search("uncultured", e, re.I) else True)

speciesGoodname = set(df_names[df_names["good"]]["id"])

speciesGoodLineageName = species & speciesGoodname

df_lineageFilter["select"] = df_lineageFilter["kingdom"].apply(lambda x: x in {"2157","2","12884","10239","4751"})
speciesKingdom = set(df_lineageFilter[df_lineageFilter["select"]]["species"])
speciesGoodLineageNameKingdom = speciesGoodLineageName & speciesKingdom

accessionGood = []
f = "/mnt/d/linux/W/NCBI/nt/nt.taxa"
fout = open("/mnt/d/linux/W/NCBI/nt_ref/nt_micro.taxa","w")
for line in open(f):
    _acc, _species = line.split()
    if _species in speciesGoodLineageNameKingdom:
        accessionGood.append(_acc)
        fout.write(_acc + "\t" + _species+"\n")
accessionGoodst = set(accessionGood)
fout.close()

#get fasta with acc in accessionGoodst

f = "/mnt/d/linux/W/NCBI/nt/nt"
fout = open("/mnt/d/linux/W/NCBI/nt_ref/nt_micro","w")
for _s in SeqIO.parse(open(f),format="fasta"):
    if _s.id.split(".")[0] in accessionGoodst:
        fout.write(">"+_s.description+"\n"+str(_s.seq) + "\n")
fout.close()

# seq length
fout = open("/mnt/d/linux/W/NCBI/nt_ref/nt_micro.len","w")
with open("/mnt/d/linux/W/NCBI/nt_ref/nt_micro") as f:
    for _s in SeqIO.parse(f,format="fasta"):
        fout.write(_s.id +"\t" + str(len(_s.seq))+"\n")
fout.close()

# goodLineage
df_lineage["keep"] = df_lineage["species"].apply(lambda x:x in speciesGoodLineageNameKingdom)
df_lineageKeep = df_lineage[df_lineage["keep"]]
print(df_lineageKeep.shape)

file_name = "/mnt/d/linux/W/NCBI/taxonamy/names.dmp"
import pandas as pd
df_name = pd.read_csv(file_name,sep = "\t", header = None,dtype = str)
df_name = df_name.loc[:,[0,2,6]]
df_name_keep = df_name[df_name[6] == "scientific name"]
dcName = dict(zip(df_name_keep[0],df_name_keep[2]))


df_lineageKeepName = df_lineageKeep.copy()
df_lineageKeepName = df_lineageKeepName.drop(labels = ["keep", "taxaCount"],axis=1)
df_lineageKeepName = df_lineageKeepName.applymap(lambda x:dcName[x] if x in dcName else "")
df_lineageKeepName["taxID"] = df_lineageKeep["species"]

df_lineageKeepName.to_csv("/mnt/d/linux/W/NCBI/nt_ref/nt_micro.goodLineage",columns = ["taxID"] + list(df_lineageKeepName.columns[:-1]), sep="\t",index = None)



```

### build database for nt_micro

```
cd /mnt/d/linux/W/NCBI/nt_ref/
/mnt/d/linux/P/blast/bin/dustmasker -in nt_micro -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out nt_micro.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in nt_micro -input_type fasta -dbtype nucl -mask_data nt_micro.asnb -out nt_micro -title "nt_micro" -parse_seqids -taxid_map nt_micro.taxa
```


## nr database

download nr from ncbi

```
l = ["/mnt/d/linux/.aspera/connect/bin/ascp -T -k 1 -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/blast/db/nr.%02d.tar.gz  ~/W/NCBI/nr201804/"%i for i in range(85)]
import subprocess
for commandline in l:
    subprocess.run(commandline,shell=True)
```

## VFDB

VFDB is virulence factor database.

### VFDB nt, get accession numbers and taxid

blast against nt

```
/mnt/d/linux/P/blast/bin/blastn -db /mnt/d/linux/W/NCBI/nt_ref/nt_micro -query /mnt/d/linux/W/NCBI/VFDB/VFDB_setB_nt.fas -out /mnt/d/linux/W/NCBI/VFDB/VFDB_setB_nt.tab -num_threads 8 -max_target_seqs 10 -word_size 56 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'


```



### VFDB protein, build blast database

rename proteins
```
from Bio import SeqIO
filename = '/mnt/d/linux/W/NCBI/VFDB/VFDB_setB_pro.fas'
fileout = '/mnt/d/linux/W/NCBI/VFDB/VFDB_blast201806/VFDB_Pr'
lis_pr = list(SeqIO.parse(open(filename),'fasta'))
fout = open(fileout,'w')
# use VFG id as seq name, and extract other information
for _l in lis_pr:
    _name = _l.id[:9] +' '+ _l.description.split(' ',1)[1]
    fout.write('>'+_name+'\n'+str(_l.seq)+'\n')
fout.close()
```
build database for blast search
```
cd /mnt/d/linux/W/NCBI/VFDB/VFDB_blast201806
/mnt/d/linux/P/blast/bin/dustmasker -in VFDB_Pr -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out VFDB_Pr.asnb

/mnt/d/linux/P/blast/bin/makeblastdb -in VFDB_Pr -input_type fasta -dbtype prot -mask_data VFDB_Pr.asnb -out VFDB_Pr -title "VFDB_Pr" -parse_seqids 
```

extract information, VF id, from the file
```
from Bio import SeqIO
import re
filename = '/mnt/d/linux/W/NCBI/VFDB/VFDB_blast201806/VFDB_Pr'
file_head = filename +'.head'
file_VFs = filename + '.VF'
lis = list(SeqIO.parse(open(filename),'fasta'))

fout = open(file_head,'w')
fout.write('VFDB_id\tVFDB_description\n')
for _l in lis:
    fout.write('\t'.join(_l.description.split(' ',1)) + '\n')
fout.close()

fout = open(file_VFs, 'w')
fout.write('VFDB_id\tVF_id\n')
for _l in lis:
    _VF = re.findall('\(C*VF\d*\)',_l.description)
    if len(_VF)>0:
        _VF = _VF[0]
        _VF = _VF[1:-1]
        fout.write(_l.id+'\t'+_VF+'\n')
fout.close()
```